In [2]:
pip install -q datasets


     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 46.5 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 212 kB 49.6 MB/s 
     |████████████████████████████████| 136 kB 51.0 MB/s 
     |████████████████████████████████| 127 kB 38.6 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 49.7 MB/s 
     |████████████████████████████████| 144 kB 51.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
pip install -q transformers

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 6.5 MB 11.7 MB/s 
     |████████████████████████████████| 895 kB 39.6 MB/s 


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.figure_factory as ff
from transformers import TFAutoModel
from transformers import TrainingArguments


In [5]:
# https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'NLP_train.csv', 'test': 'NLP_test.csv'}, encoding = "ISO-8859-1")


Using custom data configuration default-84727dd1371bc16e


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-84727dd1371bc16e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
type(dataset)
dataset['train'][5]

{'fixed_text': 'people who will cure lyme disease are the patients themselves sign up today mylymedata',
 'id': 785631171531112449,
 'keywords_label': 1,
 'text': 'People who will cure #Lyme disease are the patients themselves! Sign up today! #MyLymeData https://t.co/QpxHri4Uak https://t.co/HGZTMHz6bs'}

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'keywords_label', 'fixed_text'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['id', 'text', 'keywords_label', 'fixed_text'],
        num_rows: 7500
    })
})

In [13]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")



Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
tokenizer("hello my world")

{'input_ids': [0, 2279, 23, 220, 2], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [15]:
def transform_labels(label):

    num = label['keywords_label']
    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['fixed_text'], padding='max_length')

dataset = dataset.map(tokenize_data, batched=True)

remove_columns = ['id','text','keywords_label','fixed_text']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/70000 [00:00<?, ?ex/s]

  0%|          | 0/7500 [00:00<?, ?ex/s]

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7500
    })
})

In [17]:

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base")


Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

In [19]:
# Shuffle data and make train dataset with 60,000 tweets and evaluation dataset with 10,000 tweets
temp_ds = dataset['train'].shuffle(seed=111)
train_dataset = temp_ds.select(range(60000))
eval_dataset = temp_ds.select(range(60000, 70000))


Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-84727dd1371bc16e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3495b751b8c268f0.arrow


In [20]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 60000
})

In [21]:
eval_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [22]:
dataset['train']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 70000
})

In [23]:
type(dataset['train'])

datasets.arrow_dataset.Dataset

In [24]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 60000
})

In [25]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [26]:
batch_size=64
epochs=1
training_args = TrainingArguments("test_trainer", num_train_epochs=epochs, 
                  learning_rate=2e-5, per_device_train_batch_size=batch_size, 
                  per_device_eval_batch_size=batch_size, weight_decay=0.01,
                  evaluation_strategy="epoch")

In [27]:
from sklearn.metrics import classification_report, precision_score, \
    recall_score, f1_score, accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [35]:

trainer = Trainer(
    model=model, args=training_args, compute_metrics=compute_metrics,
    train_dataset=train_dataset, eval_dataset=eval_dataset, tokenizer=tokenizer
)
trainer.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 60000
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 938


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.204500,0.131082,0.957600,0.957111,0.974861,0.939996


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
added tokens file saved in test_trainer/checkpoint-500/added_tokens.json
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=938, training_loss=0.1705964745234833, metrics={'train_runtime': 3027.0956, 'train_samples_per_second': 19.821, 'train_steps_per_second': 0.31, 'total_flos': 3946665830400000.0, 'train_loss': 0.1705964745234833, 'epoch': 1.0})

In [36]:
pred_output = trainer.predict(eval_dataset)
pred_output.metrics

***** Running Prediction *****
  Num examples = 10000
  Batch size = 64


{'test_accuracy': 0.9576,
 'test_f1': 0.9571110661541575,
 'test_loss': 0.13108214735984802,
 'test_precision': 0.9748609107768391,
 'test_recall': 0.9399960262269025,
 'test_runtime': 182.4683,
 'test_samples_per_second': 54.804,
 'test_steps_per_second': 0.86}

In [38]:
test_dataset = dataset['test']
pred_output_test = trainer.predict(test_dataset)
pred_output_test.metrics

***** Running Prediction *****
  Num examples = 7500
  Batch size = 64


{'test_accuracy': 0.9057333333333333,
 'test_f1': 0.9017919155438254,
 'test_loss': 0.2969529926776886,
 'test_precision': 0.9547058823529412,
 'test_recall': 0.8544353777309819,
 'test_runtime': 136.4035,
 'test_samples_per_second': 54.984,
 'test_steps_per_second': 0.865}

In [37]:
# y_preds = np.argmax(pred_output.predictions, axis=1)
